In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_path = 'Bike-Sharing-Dataset/hour.csv'

rides = pd.read_csv(data_path)

In [3]:
rides.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)
    
fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 
                  'weekday', 'atemp', 'mnth', 'workingday', 'hr']
data = rides.drop(fields_to_drop, axis=1)
data.head()

,yr,holiday,temp,hum,windspeed,casual,registered,cnt,season_1,season_2,...,hr_21,hr_22,hr_23,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,0,0.24,0.81,0.0,3,13,16,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0.22,0.80,0.0,8,32,40,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0.22,0.80,0.0,5,27,32,1,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0.24,0.75,0.0,3,10,13,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0.24,0.75,0.0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
quant_features = ['casual', 'registered', 'cnt', 'temp', 'hum', 'windspeed']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std
    

In [6]:
# Save data for approximately the last 21 days 
test_data = data[-21*24:]

# Now remove the test data from the data set 
data = data[:-21*24]

# Separate the data into features and targets
target_fields = ['cnt', 'casual', 'registered']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [7]:
# Hold out the last 60 days or so of the remaining data as a validation set
train_features, train_targets = features[:-60*24], targets[:-60*24]
val_features, val_targets = features[-60*24:], targets[-60*24:]

In [8]:
class NeuralNetwork(object):
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Set number of nodes in input, hidden and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        
        # Initialize weights
        self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5, 
                                       (self.input_nodes, self.hidden_nodes))

        self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                       (self.hidden_nodes, self.output_nodes))
        self.lr = learning_rate
        
        #### TODO: Set self.activation_function to your implemented sigmoid function ####
        self.activation_function = lambda x : 1/(1+np.exp(-x))
        #     def sigmoid(self, x):
        #         return 1/(1+np.exp(-x))
        #     self.activation_function = sigmoid
        
    def train(self, features, targets):
        n_records = features.shape[0]
        
        delta_weights_i_h = np.zeros(self.weights_input_to_hidden.shape)
        delta_weights_h_o = np.zeros(self.weights_hidden_to_output.shape)
        
        for X, y in zip(features, targets):
            
#             print('X is: ' + str(X))
#             print('X.shape is: ' + str(X.shape))
            #### Implement the forward pass here ####
            ### Forward pass ###
            # TODO: Hidden layer - Replace these values with your calculations.
            
            hidden_inputs = np.dot(X, self.weights_input_to_hidden) ## verified
            hidden_outputs = self.activation_function(hidden_inputs) ## verified
#             print('hidden_inputs is: ' + str(hidden_inputs))
#             print('hidden_outputs is: ' + str(hidden_outputs));
            
            # TODO: Output layer - Replace these values with your calculations.
#             final_inputs = self.activation_function(np.dot(hidden_outputs, self.weights_hidden_to_output))

            # signals into final output layer
            final_inputs = np.dot(hidden_outputs,self.weights_hidden_to_output) 
            final_outputs = final_inputs ## verified
            
#             print('final_inputs is: ' + str(final_inputs))
#             print('final_outputs is: ' + str(final_outputs))
                                                         
            #### Implement the backward pass here ####
            ### Backward pass ###
            
            # TODO: Output error - Replace this value with your calculations.
            error = y - final_outputs
#             error = final_outputs - y
#             print('error is: ' + str(error))
            
            # TODO: Calculate error term for the output unit
            output_error_term = error * 1  ## verified
#             print('output_error_term is: ' + str(output_error_term))
#             print('self.weights_hidden_to_output is: ' + str(self.weights_hidden_to_output))

#             print('self.weights_hidden_to_output.shape is: ' + str(self.weights_hidden_to_output.shape))
            # TODO: Calculate the hidden layer's contribution to the error
#             hidden_error = np.dot(output_error_term, self.weights_hidden_to_output)
#             hidden_error = np.dot(delta_weights_h_o,output_error_term)
#             hidden_error = np.dot(output_error_term, self.weights_hidden_to_output.T)
            hidden_error = np.dot(self.weights_hidden_to_output, output_error_term)
#             print('hidden_error is: ' + str(hidden_error))
            
            # TODO: Calculate the error term for the hidden layer
            hidden_error_term = hidden_error * hidden_outputs * (1 - hidden_outputs) ## verified
#             print('hidden_error_term is: ' + str(hidden_error_term))
            
#             # Weight step (input to hidden)
#            delta_weights_i_h += hidden_error_term.T * X[:, None]
#            # Weight step (hidden to output)
#            delta_weights_h_o += output_error_term * hidden_outputs[:, None]
        
            # Weight step (input to hidden)
#             delta_weights_i_h += hidden_error_term * X[:, None]
#             delta_weights_i_h += hidden_error_term * X
            delta_weights_i_h += hidden_error_term.T * X[:, None]  ## verified
#             print('delta_weights_i_h is: ' + str(delta_weights_i_h))
            # Weight step (hidden to output)
#             delta_weights_h_o += output_error_term * hidden_outputsg
            delta_weights_h_o += output_error_term * hidden_outputs[:, None] ## verified
            
            # TODO: Update the weights - Replace these values with your calculations. 
            # update hidden-to-output weights with gradient descent step
            self.weights_hidden_to_output += self.lr * delta_weights_h_o / n_records
            # update input-to-hidden weights with gradient descent step
            self.weights_input_to_hidden += self.lr * delta_weights_i_h / n_records
            
    def run(self, features):
                
        #### Implement the forward pass here ####
        
        for X in zip(features):
            
            #### Implement the forward pass here ####
            ### Forward pass ###
            # TODO: Hidden layer - replace these values with the appropriate calculations.
            # signals into hidden layer
            hidden_inputs = np.dot(X, self.weights_input_to_hidden) ## verified
            # signals from hidden layer
            hidden_outputs = self.activation_function(hidden_inputs) ## verified
            # TODO: Output layer - Replace these values with the appropriate calculations.
            # signals from final output layer 
#             final_inputs = self.activation_function(np.dot(hidden_outputs, self.weights_hidden_to_output))
            final_inputs = np.dot(hidden_outputs,self.weights_hidden_to_output) 
            # signals into final output layer
            final_outputs = final_inputs
            
            return final_outputs
        
            
            
                                   
                                   
            

In [9]:
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [10]:
import unittest

inputs = np.array([[0.5, -0.2, 0.1]])
print ('inputs.shape: ' + str(inputs.shape))
targets = np.array([[0.4]])
print ('targets.shape:' + str(targets.shape))
test_w_i_h = np.array([[0.1, -0.2],
                       [0.4, 0.5],
                       [-0.3, 0.2]])
test_w_h_o = np.array([[0.3],
                       [-0.1]])

class TestMethods(unittest.TestCase):
    
    ##########
    # Unit tests for data loading
    ##########
    
    def test_data_path(self):
        # Test that file path to dataset has been unaltered
        self.assertTrue(data_path.lower() == 'bike-sharing-dataset/hour.csv')
        
    def test_data_loaded(self):
        # Test that data frame loaded
        self.assertTrue(isinstance(rides, pd.DataFrame))
    
    ##########
    # Unit tests for network functionality
    ##########

    def test_activation(self):
        network = NeuralNetwork(3, 2, 1, 0.5)
        # Test that the activation function is a sigmoid
        self.assertTrue(np.all(network.activation_function(0.5) == 1/(1+np.exp(-0.5))))

    def test_train(self):
        # Test that weights are updated correctly on training
        network = NeuralNetwork(3, 2, 1, 0.5)
        network.weights_input_to_hidden = test_w_i_h.copy()
        network.weights_hidden_to_output = test_w_h_o.copy()
        
        network.train(inputs, targets)
        self.assertTrue(np.allclose(network.weights_hidden_to_output, 
                                    np.array([[ 0.37275328], 
                                              [-0.03172939]])))
        self.assertTrue(np.allclose(network.weights_input_to_hidden,
                                    np.array([[ 0.10562014, -0.20185996], 
                                              [0.39775194, 0.50074398], 
                                              [-0.29887597, 0.19962801]])))

    def test_run(self):
        # Test correctness of run method
        network = NeuralNetwork(3, 2, 1, 0.5)
        network.weights_input_to_hidden = test_w_i_h.copy()
        network.weights_hidden_to_output = test_w_h_o.copy()

        self.assertTrue(np.allclose(network.run(inputs), 0.09998924))

suite = unittest.TestLoader().loadTestsFromModule(TestMethods())
unittest.TextTestRunner().run(suite)

.....

inputs.shape: (1, 3)
targets.shape:(1, 1)



----------------------------------------------------------------------
Ran 5 tests in 0.032s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [11]:
import sys

### Set the hyperparameters here ###
iterations = 100
learning_rate = 0.1
hidden_nodes = 2
output_nodes = 1

N_i = train_features.shape[1]
network = NeuralNetwork(N_i, hidden_nodes, output_nodes, learning_rate)

losses = {'train':[], 'validation':[]}

for ii in range(iterations):
    # Go through a random batch of 128 records from the training data set
    batch = np.random.choice(train_features.index, size=128)
    X, y = train_features.loc[batch].values, train_targets.iloc[batch]['cnt']
    
    network.train(X, y)
    
    # Printing out the training progress
    MSE(network.run(train_features).T, train_targets['cnt'].values)


TypeError: Cannot cast array data from dtype('float64') to dtype('<U32') according to the rule 'safe'